# Gennerating data for Parser

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import json
import re

from vietnamadminunits.parser.utils import key_normalize

import warnings
warnings.filterwarnings('ignore')

BASE_DIR = Path().resolve().parent.parent

In [2]:
def create_sort(text, level=1):
    if isinstance(text, str):
        if level == 1:
            text = re.sub(r'^Tỉnh\s|Thành phố\s|Thủ đô\s', '', text, flags=re.IGNORECASE)
        else:
            text = re.sub(r'^Phường\s|Đặc khu\s|Xã\s', '', text, flags=re.IGNORECASE)

        return text.strip()
    return text


def create_keywords(row, level=1):

    ward_type_acronym = {
        'Phường': 'p',
        'Đặc khu': 'dk',
        'Xã': 'x'
    }

    keywords = []
    if level == 1:
        keywords.append(row['provinceKey'])
        keywords.append(row['provinceShortKey'])
        if pd.notnull(row['provinceAlias']):
            aliases = json.loads(row['provinceAlias'])
            for a in aliases:
                keywords.append(key_normalize(a))

    else:
        keywords.append(row['wardKey'])
        if not row['wardShortKeyDuplicated']:
            keywords.append(row['wardShortKey'])
        else:
            keywords.append(key_normalize(f"{row['wardShortKey']} {row['wardType']}"))
            keywords.append(key_normalize(f"{ward_type_acronym[row['wardType']]} {row['wardShortKey']}"))

        if pd.notnull(row['wardAlias']):
            aliases = json.loads(row['wardAlias'])
            for a in aliases:
                keywords.append(key_normalize(a))

    keywords = list(set(keywords))
    keywords = sorted(keywords, key=len, reverse=True)
    return json.dumps(keywords)


def zill_code(value, level=1):
    if not pd.isnull(value):
        if level == 1:
            return str(int(value)).zfill(2)
        elif level == 2:
            return str(int(value)).zfill(5)

    return value

In [3]:
df = pd.read_csv(BASE_DIR / 'data/processed/2025_34-province-3221-ward_with_location.csv')

In [4]:
df

,province,ward,provinceShort,wardShort,wardType,provinceCode,provinceLat,provinceLon,wardCode,wardLat,wardLon,wardAreaKm2
0,Thành phố Hà Nội,Phường Hồng Hà,Hà Nội,Hồng Hà,Phường,1,21.0001,105.698,97,21.05670,105.845,15.09
1,Thành phố Hà Nội,Phường Ba Đình,Hà Nội,Ba Đình,Phường,1,21.0001,105.698,4,21.03860,105.838,2.97
2,Thành phố Hà Nội,Phường Ngọc Hà,Hà Nội,Ngọc Hà,Phường,1,21.0001,105.698,8,21.03810,105.816,2.68
3,Thành phố Hà Nội,Phường Giảng Võ,Hà Nội,Giảng Võ,Phường,1,21.0001,105.698,25,21.02750,105.814,2.60
4,Thành phố Hà Nội,Phường Hoàn Kiếm,Hà Nội,Hoàn Kiếm,Phường,1,21.0001,105.698,70,21.03200,105.850,1.91
...,...,...,...,...,...,...,...,...,...,...,...,...
3316,Tỉnh Cà Mau,Xã Phú Tân,Cà Mau,Phú Tân,Xã,96,9.1362,105.182,32218,8.92852,104.846,101.70
3317,Tỉnh Cà Mau,Xã Nguyễn Việt Khái,Cà Mau,Nguyễn Việt Khái,Xã,96,9.1362,105.182,32227,8.86914,104.930,129.90
3318,Tỉnh Cà Mau,Xã Tân Ân,Cà Mau,Tân Ân,Xã,96,9.1362,105.182,32236,8.72769,105.078,218.30
3319,Tỉnh Cà Mau,Xã Phan Ngọc Hiển,Cà Mau,Phan Ngọc Hiển,Xã,96,9.1362,105.182,32244,8.64616,104.943,237.70


## Enriching data

### Adding basic columns

In [5]:
unit_cols = ['province', 'ward']
level_map = {
    'province': 1,
    'ward': 2
}

for col in unit_cols:
    # Create short version
    level = level_map[col]
    # df[f"{col}Short"] = df[col].apply(create_sort, args=(level,)) # existing

    df[f"{col}Code"] = df[f"{col}Code"].apply(zill_code, args=(level,))

    # Create key
    df[f"{col}Key"] = df[f"{col}"].apply(key_normalize)

    # Create short key
    df[f"{col}ShortKey"] = df[f"{col}Short"].apply(key_normalize)

### Checking duplication

In [6]:
# wardKey
count_ward_key = df.groupby(['province', 'wardKey']).size().reset_index(name='count').sort_values(by=['count'], ascending=False)
duplicated_ward_key = count_ward_key[count_ward_key['count']>1].copy()
duplicated_ward_key['wardKeyDuplicated'] = True
duplicated_ward_key.drop(columns=['count'], inplace=True)

print(duplicated_ward_key.shape[0])
duplicated_ward_key

# Có wardKey bị duplicated khi bỏ dấu tiếng Việt

8


,province,wardKey,wardKeyDuplicated
3299,Tỉnh Đồng Tháp,xatanthanh,True
2084,Tỉnh Quảng Ngãi,xabato,True
542,Thành phố Hồ Chí Minh,xathanhan,True
2140,Tỉnh Quảng Ngãi,xasonha,True
2829,Tỉnh Tây Ninh,xatanthanh,True
325,Thành phố Hải Phòng,xacamgiang,True
3168,Tỉnh Đồng Nai,xalocthanh,True
2618,Tỉnh Thái Nguyên,xavanlang,True


In [7]:
# Add wardKeyDuplicated
df = pd.merge(df, duplicated_ward_key, on=['province', 'wardKey'], how='left')
df['wardKeyDuplicated'].fillna(False, inplace=True)

# Đưa wardKey và wardShortKey về phiên bản có dấu tiếng Việt
df['wardKey'] = np.where(df['wardKeyDuplicated']==True, df['ward'].apply(key_normalize, args=([], False)), df['wardKey'])
df['wardShortKey'] = np.where(df['wardKeyDuplicated']==True, df['wardShort'].apply(key_normalize, args=([], False)), df['wardShortKey'])

# Preview
df[df['wardKeyDuplicated']][['province', 'ward']].sort_values(by=['province', 'ward'])

,province,ward
1059,Thành phố Hải Phòng,Xã Cẩm Giang
1060,Thành phố Hải Phòng,Xã Cẩm Giàng
2557,Thành phố Hồ Chí Minh,Xã Thanh An
2766,Thành phố Hồ Chí Minh,Xã Thạnh An
1998,Tỉnh Quảng Ngãi,Xã Ba Tô
1992,Tỉnh Quảng Ngãi,Xã Ba Tơ
1969,Tỉnh Quảng Ngãi,Xã Sơn Hà
1970,Tỉnh Quảng Ngãi,Xã Sơn Hạ
287,Tỉnh Thái Nguyên,Xã Văn Lang
671,Tỉnh Thái Nguyên,Xã Văn Lăng


In [8]:
# wardShortKey
count_ward_short_key = df.groupby(['province', 'wardShortKey']).size().reset_index(name='count').sort_values(by=['count'], ascending=False)
duplicated_ward_short_key = count_ward_short_key[count_ward_short_key['count']>1].copy()
duplicated_ward_short_key['wardShortKeyDuplicated'] = True
duplicated_ward_short_key.drop(columns=['count'], inplace=True)

print(duplicated_ward_short_key.shape[0])
duplicated_ward_short_key

# Có wardShortKey bị duplicated khi bỏ dấu tiếng Việt

2


,province,wardShortKey,wardShortKeyDuplicated
2398,Tỉnh Thanh Hóa,dongtien,True
3269,Tỉnh Đồng Tháp,mytho,True


In [9]:
# Add wardShortKeyDuplicated
df = pd.merge(df, duplicated_ward_short_key, on=['province', 'wardShortKey'], how='left')
df['wardShortKeyDuplicated'].fillna(False, inplace=True)
df[df['wardShortKeyDuplicated']==True]

# Cần bổ sung dạng DICT mà wardKey là no accented mà shortKey là accented

,province,ward,provinceShort,wardShort,wardType,provinceCode,provinceLat,provinceLon,wardCode,wardLat,wardLon,wardAreaKm2,provinceKey,provinceShortKey,wardKey,wardShortKey,wardKeyDuplicated,wardShortKeyDuplicated
1372,Tỉnh Thanh Hóa,Phường Đông Tiến,Thanh Hóa,Đông Tiến,Phường,38,20.0468,105.317,15853,19.8511,105.727,41.97,tinhthanhhoa,thanhhoa,phuongdongtien,dongtien,False,True
1472,Tỉnh Thanh Hóa,Xã Đồng Tiến,Thanh Hóa,Đồng Tiến,Xã,38,20.0468,105.317,15724,19.7680,105.679,19.96,tinhthanhhoa,thanhhoa,xadongtien,dongtien,False,True
2828,Tỉnh Đồng Tháp,Phường Mỹ Tho,Đồng Tháp,Mỹ Tho,Phường,82,10.4293,105.999,28261,10.3501,106.375,6.40,tinhdongthap,dongthap,phuongmytho,mytho,False,True
3035,Tỉnh Đồng Tháp,Xã Mỹ Thọ,Đồng Tháp,Mỹ Thọ,Xã,82,10.4293,105.999,30076,10.4458,105.718,61.50,tinhdongthap,dongthap,xamytho,mytho,False,True


## Creating list of keywords

In [10]:
# -- CREATE ALIAS, phải làm sau khi đã fix duplicate keys

# Create alias columns with nan value
for col in ['province', 'ward']:
    df[f"{col}Alias"] = np.nan

df_province_alias = pd.read_csv(BASE_DIR / 'data/alias_keywords/from_2025/alias_province.csv')
df_ward_alias = pd.read_csv(BASE_DIR / 'data/alias_keywords/from_2025/alias_ward.csv')

province_alias_map = (
    df_province_alias
    .groupby('province_key')['alias_keyword']
    .apply(list)
    .apply(json.dumps)
    .to_dict()
)

df['provinceAlias'] = df['provinceKey'].map(province_alias_map)


ward_alias_map = (
    df_ward_alias
    .groupby(['province_key', 'ward_key'])['alias_keyword']
    .apply(list)
    .apply(json.dumps)
    .to_dict()
)

df['wardAlias'] = df.apply(
    lambda row: ward_alias_map.get((row['provinceKey'], row['wardKey'])),
    axis=1
)

In [11]:
# Create keywords
for col in unit_cols:
    level = level_map[col]
    df[f"{col}Keywords"] = df.apply(lambda row: create_keywords(row, level=level), axis=1)

## Creating dictionaries

In [12]:
# Province map
df_province = df[['provinceKey', 'provinceKeywords', 'province', 'provinceShort', 'provinceLat', 'provinceLon', 'provinceCode']].drop_duplicates().reset_index(drop=True)
DICT_PROVINCE = {}
for _, row in df_province.iterrows():
    DICT_PROVINCE[row['provinceKey']] = {
        'provinceKeywords': json.loads(row['provinceKeywords']),
        'province': row['province'],
        'provinceShort': row['provinceShort'],
        'provinceLat': row['provinceLat'],
        'provinceLon': row['provinceLon'],
        'provinceCode': row['provinceCode'],
    }

In [13]:
def build_province_ward_dict(df, short_name_key=False):
    result = {}

    # Nhóm theo provinceKey
    grouped = df.groupby('provinceKey')

    for province_key, group in grouped:
        ward_dict = {}

        for _, row in group.iterrows():
            ward_key = row['wardKey']
            if short_name_key:
                ward_keywords = [key_normalize(row['wardShort'], decode=False)]
            else:
                ward_keywords = json.loads(row['wardKeywords']) if pd.notnull(row['wardKeywords']) else []
            ward_dict[ward_key] = {
                'wardKeywords': ward_keywords,
                'ward': row['ward'],
                'wardType': row['wardType'],
                'wardShort': row['wardShort'],
                'wardLat': row['wardLat'],
                'wardLon': row['wardLon'],
                'wardCode': row['wardCode'],
            }

        result[province_key] = ward_dict

    return result

df_ward_no_accented = df[df['wardKeyDuplicated'] == False]
df_ward_accented = df[df['wardKeyDuplicated'] == True]

DICT_PROVINCE_WARD_NO_ACCENTED = build_province_ward_dict(df_ward_no_accented)
DICT_PROVINCE_WARD_ACCENTED = build_province_ward_dict(df_ward_accented)

df_ward_short_accented = df[df['wardShortKeyDuplicated']]
DICT_PROVINCE_WARD_SHORT_ACCENTED = build_province_ward_dict(df_ward_short_accented, short_name_key=True)

In [14]:
# Lấy danh sách các wardShortKey không trùng với bất kỳ hàng nào khác (trừ chính nó) và không trùng provinceShortKey
province_short_keys = set(df['provinceShortKey'].unique())
ward_short_key_counts = df['wardShortKey'].value_counts()

# Điều kiện wardUnique: xuất hiện đúng 1 lần và không nằm trong province_short_keys
df['wardUnique'] = df['wardShortKey'].map(ward_short_key_counts) == 1
df['wardUnique'] &= ~df['wardShortKey'].isin(province_short_keys)

# Chia nhỏ theo wardKeyDuplicated
df_ward_unique = df[df['wardUnique']]
df_ward_unique_accented = df_ward_unique[df_ward_unique['wardKeyDuplicated']]
df_ward_unique_no_accented = df_ward_unique[~df_ward_unique['wardKeyDuplicated']]

# Tạo dict bằng dictionary comprehension
DICT_UNIQUE_WARD_PROVINCE_NO_ACCENTED = {
    row['wardKey']: {
        'wardKeywords': json.loads(row['wardKeywords']),
        'provinceKey': row['provinceKey']
    }
    for _, row in df_ward_unique_no_accented.iterrows()
}

DICT_UNIQUE_WARD_PROVINCE_ACCENTED = {
    row['wardKey']: {
        'wardKeywords': json.loads(row['wardKeywords']),
        'provinceKey': row['provinceKey']
    }
    for _, row in df_ward_unique_accented.iterrows()
}

## Saving package data

In [15]:
# DICT
pickle_data = {
    'DICT_PROVINCE': DICT_PROVINCE,
    'DICT_PROVINCE_WARD_NO_ACCENTED': DICT_PROVINCE_WARD_NO_ACCENTED,
    'DICT_PROVINCE_WARD_ACCENTED': DICT_PROVINCE_WARD_ACCENTED,
    'DICT_UNIQUE_WARD_PROVINCE_NO_ACCENTED': DICT_UNIQUE_WARD_PROVINCE_NO_ACCENTED,
    'DICT_UNIQUE_WARD_PROVINCE_ACCENTED': DICT_UNIQUE_WARD_PROVINCE_ACCENTED,
    'DICT_PROVINCE_WARD_SHORT_ACCENTED': DICT_PROVINCE_WARD_SHORT_ACCENTED
}

with open(BASE_DIR / 'vietnamadminunits/data/parser_from_2025.json', 'w') as f:
    json.dump(pickle_data, f)


# SQLite
import sqlite3
with sqlite3.connect(BASE_DIR / 'vietnamadminunits/data/dataset.db') as conn:
    df.to_sql('admin_units', conn, if_exists='replace', index=False)